<h1><center> ENSF 519.01 Applied Data Scince </center></h1>
<h2><center> Final Project </center></h2>
<h3><center> Oscar Predictions from Movie Scripts </center></h3>
<p><center> Mark Ricalde, Paul Chen, Brian Pho, Harjee Johal <center></p>

# Phase 1

## Data Preprocessing

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(text):
    #Lower casing the text
    text = text.lower()
    
    #Removing punctuation
    translation = text.maketrans("", "", string.punctuation)
    text = text.translate(translation)
    re.sub("\s\s+", " ", text)
    
    #Removing stop words
    text = " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    
    #Stemming the words
    text = " ".join([stemmer.stem(word) for word in text.split()])

    return text


# Load dataset
data = pd.read_csv('oscar_data.csv') # TODO - Replace dummy csv with real merged csv

# Cleaning the script
data["script"] = data["script"].apply(clean_text)

# Features/Labels for classifying whether a movie will win an oscar
features_1 = data.loc[:, data.columns != 'winner']
labels_1   = data["winner"]

# Features/Labels for classifying what category a movie will be in
features_2 = data.loc[:, data.columns != 'category']
labels_2   = data["category"]
   
# Splitting the data set
xtrain, xtest, ytrain, ytest = train_test_split(features_1, labels_1, stratify=labels_1, test_size=0.2)

# Vectorizing the training set
# TODO - Only vectorize textual features
# tfidf_vect = TfidfVectorizer().fit(xtrain)
# xtrain     = tfidf_vect.transform(xtrain)


KeyError: 'script'

## Visualization and Exploratory Data Analysis

# Phase 2

## Discriminative Models

## Generative Models

## Clustering Models

## Machine Learning Models

### Get imports

In [20]:
# Common imports
import numpy as np
import pandas as pd
import tensorflow as tf

### Get data and labels

In [105]:
from keras.datasets import imdb

(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(input_train)
print(y_train)

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

In [118]:
# Get scripts data
script_csv_path = "./processed_data/script_data.csv"
script_data = pd.read_csv(script_csv_path)
script_data = script_data.drop(columns=["release_year"])
# display(script_data)

# Get Oscar labels
oscar_labels_path = "./processed_data/oscar_data.csv"
oscar_label = pd.read_csv(oscar_labels_path).rename(columns={"entity": "title", "category": "nominated", "winner": "won"})
oscar_label = oscar_label.drop(columns=["year"])
oscar_label["nominated"] = True
# display(oscar_label)

# Merge script and Oscar data
script_oscar_df = pd.merge(script_data, oscar_label, how="outer", on="title")
script_oscar_df = script_oscar_df.dropna(subset=['script'])
script_oscar_df = script_oscar_df.fillna(False)
# display(script_oscar_df)

script_oscar_df['nominated'].astype('bool')
script_oscar_df['won'].astype('bool')
# print(script_oscar_df.dtypes)
# display(script_oscar_df)

script_oscar_df = script_oscar_df[script_oscar_df['nominated'] == script_oscar_df['won']].drop_duplicates()

script_oscar_df = script_oscar_df.drop(columns=['nominated'])
script_oscar_df['won'] = script_oscar_df['won'].astype(int)
script_oscar_df = script_oscar_df.reset_index(drop=True)

display(script_oscar_df)
# print("Won:", np.unique(script_oscar_df['won'], return_counts=True))

,title,script,won
0,When Worlds Collide,"['needl', 'heaven', 'haystack', 'star', 'heave...",1
1,Reds,"['rememb', 'im', 'begin', 'forget', 'peopl', '...",1
2,Undefeated,"['let', 'start', 'right', 'guard', 'shot', 'lo...",1
3,Seven Brides for Seven Brothers,"['deliv', 'perfectionand', 'dont', 'brag', 'd'...",1
4,Henry V,"['oh', 'muse', 'ascend', 'brightest', 'heaven'...",1
...,...,...,...
3195,The Roommate,"['hi', 'hi', 'sara', 'matthew', 'sara', 'matth...",0
3196,Night of the Comet,"['record', 'time', 'swung', 'univers', 'ellipt...",0
3197,Paddington,"['darkest', 'peru', 'vast', 'unexplor', 'wilde...",0
3198,Jumanji: Welcome to the Jungle,"['ocean', 'wave', 'crash', 'seagul', 'squawk',...",0


### Preprocess data for NN

In [132]:
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.model_selection import train_test_split

vocab_size = 10000
seed = 0

script_oscar_df['one_hot_script'] = script_oscar_df['script'].apply(one_hot, n=vocab_size)
display(script_oscar_df)

X_train, X_test, y_train, y_test = train_test_split(script_oscar_df['one_hot_script'], script_oscar_df['won'], random_state=seed)

,title,script,won,one_hot_script
0,When Worlds Collide,"['needl', 'heaven', 'haystack', 'star', 'heave...",1,"[4426, 3812, 2377, 9826, 3812, 4733, 6004, 692..."
1,Reds,"['rememb', 'im', 'begin', 'forget', 'peopl', '...",1,"[2880, 3027, 9486, 2985, 2001, 9018, 310, 2880..."
2,Undefeated,"['let', 'start', 'right', 'guard', 'shot', 'lo...",1,"[1581, 1060, 7782, 4672, 8066, 6538, 2218, 106..."
3,Seven Brides for Seven Brothers,"['deliv', 'perfectionand', 'dont', 'brag', 'd'...",1,"[743, 5432, 9046, 9500, 4681, 8404, 8038, 735,..."
4,Henry V,"['oh', 'muse', 'ascend', 'brightest', 'heaven'...",1,"[5823, 7892, 7296, 4778, 3812, 2030, 9396, 348..."
...,...,...,...,...
3195,The Roommate,"['hi', 'hi', 'sara', 'matthew', 'sara', 'matth...",0,"[5817, 5817, 353, 733, 353, 733, 5129, 4893, 1..."
3196,Night of the Comet,"['record', 'time', 'swung', 'univers', 'ellipt...",0,"[8629, 4412, 3302, 8657, 9760, 1091, 2288, 939..."
3197,Paddington,"['darkest', 'peru', 'vast', 'unexplor', 'wilde...",0,"[2502, 9833, 9761, 2073, 3076, 7026, 1831, 264..."
3198,Jumanji: Welcome to the Jungle,"['ocean', 'wave', 'crash', 'seagul', 'squawk',...",0,"[3467, 4393, 4266, 1170, 8885, 4618, 1788, 413..."


In [129]:
print(X_train.shape)
print(y_train.shape)

(2400,)
(2400,)


### Create Simple RNN model

In [130]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN

model = keras.Sequential()
model.add(Embedding(vocab_size, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_16 (SimpleRNN)    (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_17 (SimpleRNN)    (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_18 (SimpleRNN)    (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 33        
Total params: 326,273
Trainable params: 326,273
Non-trainable params: 0
_________________________________________________________________


### Train Simple RNN model

In [134]:
simple_rnn_history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

### Plot results

In [32]:
import matplotlib.pyplot as plt

def plot_results(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')

    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_results(simple_rnn_history)

### Create LSTM model

In [31]:
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


### Train LSTM model

In [39]:
lstm_history = model.fit(input_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2)

### Plot results

In [ ]:
plot_results(lstm_history)